<a href="https://colab.research.google.com/github/ManojPanda3/SoS-Sign-Identifier/blob/main/SOSHandSign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F589922%2F1064771%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240902T032758Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D63d3d0b32c5beb7ff5382d3de504e7ec4b88faad75c14905714e095e4a04129d25867831b1f335f16af4ccb60ff91affdb20d14faed0c5adc5865b332aa7f530a2ea359eeb2148010d096b8490b1099eb8fa3eafe45217edc830136240ed649cf01f98d85610407146378bad0109327f3253ddf4d53b176597fcf793635b5b3b46a0d4c9a4dd52ea926cc57e14409926a10a288e09a3d90df6eaceddc4c7cab799deab4267c79d80c6811ebe2c8b461207de3d7efe6cfafea995e19fbb8a75a4e4e492d9dcaaff2a94000f16eb65b82c3a28b76aed1327ff7dd2ebf0e874470c6a771ec6b2da8e487453aed2441b04259cb176b29e14c99ef6d918ef988c073d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 24265584 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


In [39]:
from tensorflow.python.client import device_lib
import tensorflow as tf

!export CUDA_VISIBLE_DEVICES=0
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16575786769028219505
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11785141293332358396
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
Num GPUs Available:  1


In [40]:
import numpy as np
import pandas as pd
import cv2;
import os;
from glob import glob;
from sklearn.model_selection import train_test_split;
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout , Conv2D, BatchNormalization;
from sklearn.utils import shuffle;
from tensorflow.keras.utils import to_categorical;
from tensorflow.keras.applications import MobileNetV2,ResNet50
import random
from keras.callbacks import EarlyStopping;

In [41]:
ImagesPath = "/kaggle/input/Train"
ImagesType = "*"
ImageSize = (64,64,3)

This is for finding file name for other hand signs :)

In [42]:
import itertools

otherHands = list(itertools.chain(*[glob(os.path.join(i,ImagesType))[:46:2] for i in glob(os.path.join(ImagesPath,"*"), recursive=True) if i[-1]not in ("M","B")]));
random.shuffle(otherHands);
print(len(otherHands))
otherHands = otherHands[:150];
print(len(otherHands))

506
150


In [43]:
dataB = glob(os.path.join(ImagesPath+"/B",ImagesType))[:180];
dataM = glob(os.path.join(ImagesPath+"/M",ImagesType))[:180];
print(len(dataB),len(dataM))

180 180


In [44]:
dataReturns = {
    "B":1,"M":2
}
def ImageExtractor(data,size,mode=0,Y=[]):
    tmpX , tmpY = [],[];
    for i in range(len(data)):
        img = cv2.imread(data[i]);
        img_resized = cv2.resize(img, size[0:2]);
        if(mode and Y):
            angle  = random.randint(1,45);
            tmpX.append(np.array(img_resized,dtype=float) / 255.0);
            img_rotate = [np.array(cv2.warpAffine(img_resized,cv2.getRotationMatrix2D((img_resized.shape[1]/2, img_resized.shape[0]/2), angle, 1)
,(img_resized.shape[1], img_resized.shape[0])))/255.0 for i in range(int(360/angle)-1)];
            img_flip = np.array(cv2.flip(img_resized,1));
            tmpX.append(img_flip);
            tmpY.append(Y[i]);
            tmpY.append(Y[i]);
            img_flip_rotate = [np.array(cv2.warpAffine(img_flip,cv2.getRotationMatrix2D((img_flip.shape[1]/2, img_flip.shape[0]/2), angle, 1)
,(img_flip.shape[1], img_flip.shape[0])))/255.0 for i in range(int(360/(angle*2))-1)];
            tmpX.extend(img_flip_rotate);
            tmpX.extend(img_rotate);
            tmpY.extend([Y[i]]*(len(img_rotate)));
            tmpY.extend([Y[i]]*(len(img_flip_rotate)));
        else:
            data[i]=np.array(img_resized,dtype=float) / 255.0;

    if(mode and Y): return tmpX,tmpY
    return data

In [45]:
X , Y = [] , [];
X.extend(otherHands);
Y.extend([0]*len(otherHands))
X.extend(dataB);
Y.extend([dataReturns["B"]]*len(dataB))
X.extend(dataM);
Y.extend([dataReturns["M"]]*len(dataM))
print(len(X),len(Y));

510 510


In [46]:
X,Y = ImageExtractor(X,ImageSize[:2],mode=1,Y=Y)
print(len(X),len(Y))
X, Y = shuffle(X, Y, random_state=42)
X = np.array(X);
Y = to_categorical(np.array(Y),num_classes=3);

28237 28237


In [47]:
# Test Data Import
ImagesPathTest = "/kaggle/input/handsignimages/Test"
ImagesTypeTest = "*.jpg"
testY = []
testX = list(itertools.chain(*[glob(os.path.join(i,ImagesType))[:23:2] for i in glob(os.path.join(ImagesPath,"*"), recursive=True) if i[-1]not in ("M","B")]));
testY.extend([0]*len(testX));
testX.extend(glob(os.path.join(ImagesPath+"/B",ImagesType))[:23:2]);
testY.extend([dataReturns["B"]]*(len(testX)-len(testY)));
testX.extend(glob(os.path.join(ImagesPath+"/M",ImagesType))[:23:2]);
testY.extend([dataReturns["M"]]*(len(testX)-len(testY)));
testX = ImageExtractor(testX,ImageSize[:2]);
testX , testY = shuffle(testX,testY,random_state=42);
testX = np.array(testX);
testY = to_categorical(np.array(testY), num_classes=3);
print(testX.shape,testY.shape);

(288, 64, 64, 3) (288, 3)


Model

In [48]:
base_model = MobileNetV2(input_shape=ImageSize, include_top=False, weights='imagenet')
x = base_model.output;
# x = Conv2D(64, (3, 3), padding='same', activation='relu')()
# x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x);
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
layers = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input,outputs=layers)
for l in base_model.layers: l.trainable=True;
model.compile(optimizer='adamW', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=10,             # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True, # Restore model weights from the epoch with the best value of the monitored metric
     mode='min',
)
model.summary();



<ipython-input-48-c951bfbf2cfd>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=ImageSize, include_top=False, weights='imagenet')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,340,163 (8.93 MB)

 Trainable params: 2,306,051 (8.80 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [49]:
with tf.device('/device:GPU:0'):
  model.fit(X, Y, epochs=50, batch_size=64, validation_data=(testX, testY),callbacks=[early_stopping]);

Epoch 1/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 83s 89ms/step - accuracy: 0.8772 - loss: 0.3404 - val_accuracy: 0.9097 - val_loss: 1.1777
Epoch 2/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 38s 30ms/step - accuracy: 0.9763 - loss: 0.0654 - val_accuracy: 0.0417 - val_loss: 26.6380
Epoch 3/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9753 - loss: 0.0778 - val_accuracy: 0.9167 - val_loss: 2.5267
Epoch 4/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9944 - loss: 0.0185 - val_accuracy: 0.9167 - val_loss: 3.6244
Epoch 5/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.9915 - loss: 0.0275 - val_accuracy: 0.9167 - val_loss: 3.6250
Epoch 6/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9987 - loss: 0.0049 - val_accuracy: 0.9167 - val_loss: 1.7029
Epoch 7/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.9981 - loss: 0.0086 - val_accuracy: 0.9167 - val_loss: 1.0796
Epoch 8/50
442/442 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9267 - loss: 0.2358 -

In [50]:
model.summary();

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 6,952,267 (26.52 MB)

 Trainable params: 2,306,051 (8.80 MB)

 Non-trainable params: 34,112 (133.25 KB)

 Optimizer params: 4,612,104 (17.59 MB)

In [51]:
loss , acc = model.evaluate(testX,testY);
print(acc,loss);

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9177 - loss: 0.4415
0.9166666865348816 0.4281178414821625


In [52]:
# !rm SosSign.h5
from  tensorflow.keras.saving import save_model
save_model(model,"Sos_sign_v2.h5")
# model.save("SosSign.h5")

In [57]:
from IPython.display import Image,display
testFiles = ["/kaggle/input/Test/A/112_A.jpg","/kaggle/input/Test/B/932_B.jpg","/kaggle/input/Test/X/834_X.jpg","/kaggle/input/Test/M/230_M.jpg"];
for i in testFiles:
  print(i)
  display(Image(str(i)))

/kaggle/input/Test/A/112_A.jpg


/kaggle/input/Test/B/932_B.jpg


/kaggle/input/Test/X/834_X.jpg


/kaggle/input/Test/M/230_M.jpg


In [58]:
my_test = ImageExtractor(testFiles,ImageSize[:2]);
my_test = np.array(my_test);
print(my_test.shape);

(4, 64, 64, 3)


In [59]:
pred = model.predict(my_test);
for i in range(len(pred)):
  print("res for ",i+1," num pic")
  print(np.argmax(pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
res for  1  num pic
0
res for  2  num pic
0
res for  3  num pic
0
res for  4  num pic
0


In [ ]:
print([i for i in Y if i[2]]);

[array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0